<a href="https://colab.research.google.com/github/avaneeshp23/GenAIhackathon/blob/hackathon/groqcloud_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [3]:
from groq import Groq
import pandas as pd
import json


In [4]:
client = Groq(api_key=os.environ['GROQ_API_KEY'])

In [5]:
def classify_ticket(text):
    prompt = f"""
    Classify the following IT ticket text according to these instructions:

    **Department:** Choose from: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry.
    **Priority:** Determine the urgency level: low, medium, or high.
    **Language:** Identify the language code (e.g., en, de, es).  Refer to language codes for guidance.
    **Type:** Categorize the nature of the request: Incident, Request, Problem, or Change.

    Text: {text}

    Provide the output in JSON format:
    {{
    "Department": "...",
    "Priority": "...",
    "Language": "...",
    "Type": "..."
    }}
    """
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
    )
    classification_result = response.choices[0].message.content.strip()
    return classification_result

In [66]:
!git clone https://github.com/avaneeshp23/GenAIhackathon.git

fatal: destination path 'GenAIhackathon' already exists and is not an empty directory.


In [29]:
train_df = pd.read_csv('/content/GenAIhackathon/data/train.csv')

In [50]:
def classify_ticket_deepseek(text):
    prompt = f"""
    Classify the following IT ticket text according to these instructions:

    **Department:** Choose from: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry.
    **Priority:** Determine the urgency level: low, medium, or high.
    **Language:** Identify the language code (e.g., en, de, es).  Refer to language codes for guidance.
    **Type:** Categorize the nature of the request: Incident, Request, Problem, or Change.

    Text: {text}

    Provide the output in JSON format:
    {{
    "Department": "...",
    "Priority": "...",
    "Language": "...",
    "Type": "..."
    }}
    """
    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",
        messages=[{"role": "user", "content": prompt}],
    )
    #print("Response:", response)
    classification_result = response.choices[0].message.content.strip()
    try:
        json_start = classification_result.index("{")
        json_end = classification_result.rindex("}") + 1
        classification_json = json.loads(classification_result[json_start:json_end])
        #print("Classification JSON:", classification_json)  # Check the JSON content
        return classification_json
    except (json.JSONDecodeError, ValueError) as e:
        print("Failed to parse JSON response:", e)
        return None


In [32]:
train_df['text'] = train_df['ticket_subject'] + ' ' + train_df['ticket_body']
train_df = train_df[['text', 'department', 'type', 'priority', 'language']]


In [ ]:
train_df['classification_deepseek'] = train_df['text'].apply(classify_ticket_deepseek)
train_df['classification_deepseek'][0]

In [41]:
train_df['classification_deepseek'][0]

{'Department': 'Billing and Payments',
 'Priority': 'low',
 'Language': 'es',
 'Type': 'Incident'}

In [10]:
#train_df['classification'] = train_df['text'].apply(classify_ticket)


In [11]:
#train_df['classification'][0]

'Here is the classification of the IT ticket text:\n\n```\n{\n    "Department": "Billing and Payments",\n    "Priority": "medium",\n    "Language": "es",\n    "Type": "Request"\n}\n```\n\nExplanation:\n\n* **Department:** The issue is related to a billing discrepancy, so it falls under the "Billing and Payments" department.\n* **Priority:** The user is reporting an incorrect charge, which is not a critical issue that requires immediate attention, but it\'s still important to resolve in a timely manner. Therefore, the priority is set to "medium".\n* **Language:** The text is written in Spanish, so the language code is "es".\n* **Type:** The user is requesting a correction to their bill, which is a specific request for a change to their account. Therefore, the type is classified as a "Request".'

In [42]:
test_df = pd.read_csv('/content/GenAIhackathon/data/test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...


In [43]:
test_df['text'] = test_df['ticket_subject'].fillna('') + ' ' + test_df['ticket_body'].fillna('')
test_df.shape

(500, 4)

In [ ]:
# small_df = test_df[:5]
# small_df.head()
# small_df['classification'] = small_df['text'].apply(classify_ticket_deepseek)

In [ ]:
test_df['classification'] = test_df['text'].apply(classify_ticket_deepseek)

In [46]:
# def batch_process(df, batch_size=50):
#     # Split the DataFrame into batches
#     batches = [df[i:i + batch_size] for i in range(0, len(df), batch_size)]

#     # Process each batch and store results
#     results = []
#     for batch in batches:
#         batch['classification'] = batch['text'].apply(classify_ticket)
#         results.append(batch)

#     # Concatenate all batches back into a single DataFrame
#     result_df = pd.concat(results)

#     return result_df

In [ ]:
#test_df['classification'] = test_df['text'].apply(classify_ticket)
#result_df = batch_process(test_df)

In [60]:
test_df.head()

,ticket_ID,ticket_subject,ticket_body,text,classification
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand...",Urgente: problema con la funzione di condivisi...,"{'Department': 'Technical Support', 'Priority'..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe...",Abrechnungsdiskrepanzen in Google Workspace Li...,"{'Department': 'Billing and Payments', 'Priori..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr...",Problèmes avec la fonction sans fil de l'Epson...,"{'Department': 'Product Support', 'Priority': ..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques...","Gentile assistenza clienti, \n\nSpero che que...","{'Department': 'Customer Service', 'Priority':..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...,Assistenza necessaria per gli arresti anomali ...,"{'Department': 'Technical Support', 'Priority'..."


In [61]:
test_df['classification'][0]

{'Department': 'Technical Support',
 'Priority': 'High',
 'Language': 'it',
 'Type': 'Incident'}

In [52]:
# import re
# def extract_json_from_classification(classification_text):
#     try:
#         # Extract JSON part from the classification text
#         json_text = re.search(r'\{.*\}', classification_text, re.DOTALL).group()
#         data = json.loads(json_text)
#         return data
#     except (json.JSONDecodeError, AttributeError) as e:
#         logging.error(f"Error extracting JSON: {e}")
#         return None

# def extract_values_with_id(row):
#     data = extract_json_from_classification(row['classification'])
#     if data:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': data.get('Department', None),
#             'type': data.get('Type', None),
#             'priority': data.get('Priority', None),
#             'language': data.get('Language', None)
#         })
#     else:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': None,
#             'type': None,
#             'priority': None,
#             'language': None
#         })

In [62]:
def extract_values_with_id(row):
    if pd.isna(row['classification']):
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': None,
            'type': None,
            'priority': None,
            'language': None
        })

    data = row['classification']
    if isinstance(data, dict):
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': data.get('Department', None),
            'type': data.get('Type', None),
            'priority': data.get('Priority', None),
            'language': data.get('Language', None)
        })
    else:
        return pd.Series({
            'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
            'department': None,
            'type': None,
            'priority': None,
            'language': None
        })

In [63]:
predictions_df = test_df.apply(extract_values_with_id, axis=1)

In [64]:
predictions_df.head()

,ticket_ID,department,type,priority,language
0,1,Technical Support,Incident,High,it
1,2,Billing and Payments,Incident,low,de
2,3,Product Support,Incident,medium,fr
3,4,Customer Service,Request,low,it
4,5,Technical Support,Problem,high,it


In [72]:
submission_path = '/content/GenAIhackathon/data/submission.csv'
predictions_df.to_csv(submission_path, index= False)

In [73]:
submission_df = pd.read_csv(submission_path)
submission_df.head()

,ticket_ID,department,type,priority,language
0,1,Technical Support,Incident,High,it
1,2,Billing and Payments,Incident,low,de
2,3,Product Support,Incident,medium,fr
3,4,Customer Service,Request,low,it
4,5,Technical Support,Problem,high,it
